In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json

Idea:
- Part 1: Just run through the graph resolving numbers. Easy enough.
- Part 2: If the divisions are all divisions by constants, i.e. they don't depend on `humn`, then the number of each monkey depends polynomially on `humn` so let's write out the polynomials. Then we'll simply solve the polynomial equation that `root` wants us to solve. 

In [2]:
class Monkey:
    def __init__(self, line) -> None:
        if ' ' in line:
            self.number = None
            self.number2 = None
            line = line.split(' ')
            self.first, self.operator, self.second = line
            return
        self.number = int(line)
        self.number2 = [self.number,]
    
    def get_number(self, other_monkeys):
        if self.number is None:
            x = other_monkeys[self.first].get_number(other_monkeys)
            y = other_monkeys[self.second].get_number(other_monkeys)
            match self.operator:
                case '+':
                    self.number = x+y
                case '-':
                    self.number = x-y
                case '*':
                    self.number = x*y
                case '/':
                    self.number = x/y
                case _:
                    raise Exception("Invalid operator.")
        return self.number
    
    def get_number2(self, other_monkeys):
        if self.number2 is None:
            x = other_monkeys[self.first].get_number2(other_monkeys)
            y = other_monkeys[self.second].get_number2(other_monkeys)
            self.number2 = []
            match self.operator:
                case '+':
                    for i in range(max(len(x), len(y))):
                        self.number2.append(0)
                        if i < len(x):
                            self.number2[i] += x[i]
                        if i < len(y):
                            self.number2[i] += y[i]
                case '-':
                    for i in range(max(len(x), len(y))):
                        self.number2.append(0)
                        if i < len(x):
                            self.number2[i] += x[i]
                        if i < len(y):
                            self.number2[i] -= y[i]
                case '*':
                    for i in range(len(x)+len(y)-1):
                        self.number2.append(0)
                        for j in range(i+1):
                            if j >= len(x) or i-j >= len(y):
                                continue
                            self.number2[i] += x[j]*y[i-j]
                case '/':
                    if len(y) > 1 and any(y[i] != 0 for i in range(1, len(y))):
                        raise Exception("Division by non-constant polynomial. Unexpected")
                    for i in range(len(x)):
                        self.number2.append(x[i]/y[0])
                case _:
                    raise Exception("Invalid operator.")
        return self.number2

In [3]:
monkeys = {}

with open("./data/day21.txt") as f:
    while line := f.readline():
        line = line.rstrip().split(': ')
        monkeys[line[0]] = Monkey(line[1])
len(monkeys)

1619

In [4]:
int(monkeys['root'].get_number(monkeys))

158731561459602

In [5]:
monkeys['humn'].number2 = [0, 1]

In [6]:
first = monkeys[monkeys['root'].first].get_number2(monkeys)
second = monkeys[monkeys['root'].second].get_number2(monkeys)

first, second

([117768843708843.94, -20.374768089053802], [40962717833337.0])

We get the equation $-20.374768089053802*\text{humn}+117768843708843.94 = 40962717833337.0$ (with some floating point error but should be close enough) that we can solve for the answer:

In [7]:
round((second[0]-first[0])/first[1])

3769668716709